# <font color='red'> Model-3 </font>

<img src='https://i.imgur.com/fkQ8nGo.png'>
ref: https://i.imgur.com/fkQ8nGo.png

In [1]:
#in this model you can use the text vectorized data from model1 
#for other than text data consider the following steps
# you have to perform one hot encoding of categorical features. You can use onehotencoder() or countvectorizer() for the same.
# Stack up standardised numerical features and all the one hot encoded categorical features
#the input to conv1d layer is 3d, you can convert your 2d data to 3d using np.newaxis
# Note - deep learning models won't work with sparse features, you have to convert them to dense features before fitting in the model.

In [2]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Input , Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from tensorflow.keras.metrics import AUC

from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from scipy.sparse import coo_matrix, hstack
from scipy.sparse import csr_matrix
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
tf.keras.backend.clear_session()

In [3]:
#read the csv file
import pandas as pd
p1 = '/content/drive/MyDrive/AAIC/Assignments/LSTM on Donors Choose/preprocessed_data_final.csv'
p2 = "C:/Users/darsh/Downloads/Srujan/Donars Choose Assignment/preprocessed_data_final.csv"
df = pd.read_csv(p2)

In [4]:
 df[pd.isnull(df).any(axis=1)]

,teacher_number_of_previously_posted_projects,resource_summary_contains_numerical_digits,price,quantity,school_state,project_grade_category,clean_categories,clean_subcategories,teacher_prefix,project_is_approved,essay,project_title,project_resource_summary


In [5]:
y = df['project_is_approved'].values
df.drop(['project_is_approved'],axis=1,inplace=True)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y,
                                                    stratify=y, 
                                                    test_size=0.25,random_state=0)

In [7]:
X_train.head(2)

,teacher_number_of_previously_posted_projects,resource_summary_contains_numerical_digits,price,quantity,school_state,project_grade_category,clean_categories,clean_subcategories,teacher_prefix,essay,project_title,project_resource_summary
92061,6,0,234.95,4,nc,grades_3_5,music_arts,visualarts,mrs,school center wonderful community many student...,Family Time is Art Time!,"My students need a heavy duty paper cutter, pa..."
83229,47,0,324.06,6,nc,grades_6_8,math_science,mathematics,ms,teach wonderful group sixth grade students hig...,Math Lounge Part 2,My students need a chair with pillows for thei...


# <font color='blue'> Text vectorization </font>

In [8]:
text_input = ['essay','project_title','project_resource_summary',]
X_train['total_text_input'] = X_train['essay'] + ' ' + X_train['project_title'] + ' ' + X_train['project_resource_summary']
X_test['total_text_input'] = X_test['essay'] + ' ' + X_test['project_title'] + ' ' + X_test['project_resource_summary']


In [9]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

# Tokenize our training data
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(X_train['total_text_input'])

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(X_train['total_text_input'])
test_sequences = tokenizer.texts_to_sequences(X_test['total_text_input'])

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

In [10]:
# Output the results of our work
#print("Word index:\n", word_index)
#print("\nTraining sequences:\n", train_sequences)
#print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape, Test Shape:", train_padded.shape,test_padded.shape)
print("Training sequences data type:", type(train_sequences),type(test_sequences))
print("Padded Training sequences data type:", type(train_padded),type(test_padded))


Padded training shape, Test Shape: (81936, 355) (27312, 355)
Training sequences data type: <class 'list'> <class 'list'>
Padded Training sequences data type: <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [11]:
# load the whole embedding into memory
embeddings_index = dict()
p1 = '/content/drive/MyDrive/AAIC/Assignments/LSTM on Donors Choose/glove.6B.300d.txt'
p2 = "C:/Users/darsh/Downloads/Srujan/Donars Choose Assignment/glove.6B.300d.txt"
f = open(p2,encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('\nLoaded %s word vectors.' % len(embeddings_index))

400000it [00:19, 20102.10it/s]


Loaded 400000 word vectors.


In [12]:
vocab_size = len(tokenizer.word_index) + 1

In [13]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# <font color='blue'> categorical and Numerical vectorization </font>

In [14]:
#in this model you can use the text vectorized data from model1 
#for other than text data consider the following steps
# you have to perform one hot encoding of categorical features. You can use onehotencoder() or countvectorizer() for the same.
# Stack up standardised numerical features and all the one hot encoded categorical features
#the input to conv1d layer is 3d, you can convert your 2d data to 3d using np.newaxis
# Note - deep learning models won't work with sparse features, you have to convert them to dense features before fitting in the model.

In [15]:

categorical_input = ['school_state','project_grade_category','clean_categories', 'clean_subcategories','teacher_prefix',]


In [16]:
categorical_data_train = dict()
categorical_data_test = dict()
for i in categorical_input:
    vectorizer = CountVectorizer()
    train_enc = vectorizer.fit_transform(X_train[i].values)
    test_enc = vectorizer.transform(X_test[i].values)
    categorical_data_train[i+'_enc'] = train_enc#.todense()
    categorical_data_test[i+'_test_enc'] = test_enc#.todense()
    print(i,train_enc.shape,test_enc.shape)
    
    

school_state (81936, 51) (27312, 51)
project_grade_category (81936, 4) (27312, 4)
clean_categories (81936, 51) (27312, 51)
clean_subcategories (81936, 393) (27312, 393)
teacher_prefix (81936, 5) (27312, 5)


In [17]:
numerical_input = ['teacher_number_of_previously_posted_projects',
                  'resource_summary_contains_numerical_digits',
                  'price','quantity']

In [18]:
scaler  = preprocessing.MinMaxScaler().fit(X_train[numerical_input].values)
std_data_train = (pd.DataFrame(scaler.transform(X_train[numerical_input]),columns=numerical_input))
std_data_test = (pd.DataFrame(scaler.transform(X_test[numerical_input]),columns=numerical_input))

In [19]:
other_than_text_data = hstack(((std_data_train),
                               categorical_data_train['school_state_enc'],
                               categorical_data_train['project_grade_category_enc'],
                               categorical_data_train['clean_categories_enc'],
                               categorical_data_train['clean_subcategories_enc'],
                               categorical_data_train['teacher_prefix_enc'],
                               )).todense()

In [20]:
#np.array(other_than_text_data).reshape(81936,507,1)
#other_than_text_data = other_than_text_data.reshape(81936,507,1)

In [21]:
other_than_text_data = other_than_text_data[:,:,np.newaxis]

In [22]:
type(other_than_text_data.reshape(81936,508,1))

numpy.matrix

In [23]:
other_than_text_data_test = np.array(hstack(((std_data_test),
                                    categorical_data_test['school_state_test_enc'],
                                    categorical_data_test['project_grade_category_test_enc'],
                                    categorical_data_test['clean_categories_test_enc'],
                                    categorical_data_test['clean_subcategories_test_enc'],
                                    categorical_data_test['teacher_prefix_test_enc'],
                                   )).todense())

In [24]:
other_than_text_data_test = other_than_text_data_test[:,:,np.newaxis]

In [25]:
from tensorflow.keras.layers import Conv1D

input_seq_total_text_data = Input(shape=(maxlen,),name='input_seq_total_text_data_')

emb_text_data = Embedding(input_dim=vocab_size,output_dim=300, 
                          weights=[embedding_matrix], input_length=maxlen,trainable=False,
                          name='emb_text_data')(input_seq_total_text_data)
lstm = LSTM(units=128,activation='tanh',return_sequences=True)(emb_text_data)
flatten_text = Flatten()(lstm)

other_than_text_data_ = Input(shape=(508,1))

#other_than_text_data_ = Input(shape=(508,1),name='other_than_text_data1')
conv1 = Conv1D(filters=128,kernel_size=3,strides=1,)(other_than_text_data_)
conv2 = Conv1D(filters=128,kernel_size=3,strides=1,)(conv1)
flat1 = Flatten()(conv2)

concat = concatenate([flatten_text,flat1])

d1 = Dense(units=256,activation='relu',kernel_initializer='he_normal',
           kernel_regularizer=l2(0.000001),name='dense_layer_1')(concat)
drop1 = Dropout(0.5)(d1)

d2 = Dense(units=256,activation='relu',kernel_initializer='he_normal',
           kernel_regularizer=l2(0.000001),name='dense_layer_2')(drop1)
drop2 = Dropout(0.5)(d2)

bn1 = BatchNormalization()(drop2)

d3 = Dense(units=256,activation='relu',kernel_initializer='he_normal',
           kernel_regularizer=l2(0.000001),name='dense_layer_3')(bn1)
drop3 = Dropout(0.5)(d3)

d4 = Dense(units=256,activation='relu',kernel_initializer='he_normal',
           kernel_regularizer=l2(0.000001),name='dense_layer_4')(drop3)
drop4 = Dropout(0.5)(d4)



d5 = Dense(units=256,activation='relu',kernel_initializer='he_normal',
           kernel_regularizer=l2(0.000001),name='dense_layer_5')(drop4)
drop5 = Dropout(0.5)(d5)


output = Dense(units=2,activation='softmax')(drop5)

In [26]:
m3 = Model(inputs=[input_seq_total_text_data,other_than_text_data_],
           outputs=[output])

In [27]:
m3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_seq_total_text_data_ (Inp [(None, 355)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 508, 1)]     0                                            
__________________________________________________________________________________________________
emb_text_data (Embedding)       (None, 355, 300)     17031600    input_seq_total_text_data_[0][0] 
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 506, 128)     512         input_1[0][0]                    
______________________________________________________________________________________________

In [28]:
test_data = [test_padded,other_than_text_data_test]

train_data = [train_padded,other_than_text_data]             

y_train_enc =  tensorflow.keras.utils.to_categorical(y_train, 2)
y_test_enc =  tensorflow.keras.utils.to_categorical(y_test, 2)

In [29]:
def auc1(y_true, y_pred):
    if len(np.unique(y_true[:,1])) == 1:
        return 0.5
    else:
        return roc_auc_score( y_true, y_pred, average='macro', sample_weight=None).astype('double')

def auroc(y_true, y_pred):
    return tensorflow.numpy_function(auc1, (y_true, y_pred), tensorflow.double)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./LSTM_Model_3.h5', save_weights_only=False,save_best_only=True, \
                                       mode='max', monitor='val_auroc',verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auroc', patience=2,mode='max',verbose=1),
]



In [30]:
m3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[auroc])

In [31]:
steps = len(y_train_enc)//128

In [32]:
m3.fit(train_data,y_train_enc,
       validation_data=(test_data,y_test_enc),
       batch_size=128,
       epochs=50,
       callbacks=callbacks,
       verbose=1)

Epoch 1/50
641/641 [==============================] - 54s 69ms/step - loss: 0.5118 - auroc: 0.5022 - val_loss: 0.5550 - val_auroc: 0.4308

Epoch 00001: val_auroc improved from -inf to 0.43082, saving model to .\LSTM_Model_3.h5
Epoch 2/50
641/641 [==============================] - 43s 67ms/step - loss: 0.4400 - auroc: 0.5046 - val_loss: 0.5034 - val_auroc: 0.5878

Epoch 00002: val_auroc improved from 0.43082 to 0.58782, saving model to .\LSTM_Model_3.h5
Epoch 3/50
641/641 [==============================] - 43s 68ms/step - loss: 0.4362 - auroc: 0.5027 - val_loss: 0.4607 - val_auroc: 0.4182

Epoch 00003: val_auroc did not improve from 0.58782
Epoch 4/50
641/641 [==============================] - 43s 67ms/step - loss: 0.4340 - auroc: 0.4999 - val_loss: 0.4464 - val_auroc: 0.4063

Epoch 00004: val_auroc did not improve from 0.58782

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/50
641/641 [==============================] - 44s 68ms/step - loss: 0.4

641/641 [==============================] - 44s 68ms/step - loss: 0.3349 - auroc: 0.8094 - val_loss: 0.4296 - val_auroc: 0.7445

Epoch 00037: val_auroc did not improve from 0.74928
Epoch 38/50
641/641 [==============================] - 43s 68ms/step - loss: 0.3341 - auroc: 0.8076 - val_loss: 0.4291 - val_auroc: 0.7445

Epoch 00038: val_auroc did not improve from 0.74928

Epoch 00038: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-12.
Epoch 39/50
641/641 [==============================] - 43s 68ms/step - loss: 0.3349 - auroc: 0.8086 - val_loss: 0.4289 - val_auroc: 0.7445

Epoch 00039: val_auroc did not improve from 0.74928
Epoch 40/50
641/641 [==============================] - 43s 68ms/step - loss: 0.3333 - auroc: 0.8119 - val_loss: 0.4291 - val_auroc: 0.7445

Epoch 00040: val_auroc did not improve from 0.74928

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1.0000001044244145e-13.
Epoch 41/50
641/641 [==============================] - 43s 68ms/step - loss: 0.33